In [1]:
import pandas as pd
from datetime import date, timedelta

In [2]:
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [3]:
day = date(2024, 3, 1)
day_dateoffset = day - pd.DateOffset(months=1)
print(day_dateoffset)

2024-02-01 00:00:00


In [4]:
day = date(2024, 3, 1)
day_timedelta = day - timedelta(days=31)
print(day_timedelta)

2024-01-30


In [5]:
f = open('call_rate_test.csv', 'r', encoding='utf-8')

# 데이터를 완전히 동일하게 가져올 수 있는가
headers = f.readline().rstrip() # 헤더 읽기, 맨 끝 줄바꿈기호 삭제
print(headers.split('\t'))
headers = headers.split('\t') # 같은 이름의 변수에 리스트로 바꾼 것 넣어주기

# 데이터 읽기
lines = f.readlines()
print(lines[:5])

new_datas = []
for line in lines: # 한 줄씩 
    data = line.rstrip().split('\t') # 줄바꿈 기호 지우고, ,로 잘라준다.
    dict_data = { header: data[i] for i, header in enumerate(headers) }
    new_datas.append(dict_data) # 데이터를 딕셔너리로 만들어서 넣어야된다.

f.close()

['date', '종가', '등락률']
['2023-10-19\t3.5050\t0.37\n', '2023-10-18\t3.4920\t-0.02\n', '2023-10-17\t3.4930\t-0.71\n', '2023-10-16\t3.5180\t-0.67\n', '2023-10-13\t3.5420\t0.85\n']


In [6]:
def find_nearest_date(df, target_date):
    delta = pd.Timedelta(days=1)
    nearest_date = None
    min_difference = pd.Timedelta.max

    for date in df['date']:
        difference = abs(target_date - date)
        if difference < min_difference:
            min_difference = difference
            nearest_date = date

    return nearest_date

def label_call(df):
    df_ud = pd.DataFrame(index=df.index, columns=['date', '종가', 'label'])

    # '날짜' 열을 datetime 형식으로 변환
    df['date'] = pd.to_datetime(df['date'])

    for i, row in df.iterrows():
        time = row['date']
        time2 = time - pd.DateOffset(months=1)

        # 이전 달의 날짜가 인덱스에 있는지 확인
        if time2 in df['date'].values:
            prev_row = df[df['date'] == time2].iloc[0]
        else:
            # 이전 날짜가 없는 경우 가장 가까운 날짜로 설정
            nearest_date = find_nearest_date(df, time2)
            prev_row = df[df['date'] == nearest_date].iloc[0]

        if row['종가'] > prev_row['종가']:
            label = '매파'
        elif row['종가'] == prev_row['종가']:
            label = 0
        else:
            label = '비둘기파'

        df_ud.loc[i] = [time.date(), row['종가'], label]

    return df_ud

In [7]:
df = pd.DataFrame(new_datas)

result = label_call(df)
result

,date,종가,label
0,2023-10-19,3.5050,비둘기파
1,2023-10-18,3.4920,비둘기파
2,2023-10-17,3.4930,비둘기파
3,2023-10-16,3.5180,비둘기파
4,2023-10-13,3.5420,비둘기파
5,2023-10-12,3.5120,비둘기파
6,2023-10-05,3.5270,비둘기파
7,2023-10-04,3.5550,비둘기파
8,2023-09-27,3.8290,매파
9,2023-09-26,3.6200,매파


In [8]:
new_df = pd.read_json("news_test.json")
new_df = new_df.sort_values(by=["date"], ascending=False) # 날짜별 내림차순

new_df = pd.DataFrame(new_df)
new_df

,date,content
6,2023-10-19,원/달러 환율이 장중 한때 1360원에 근접했다. 중동분쟁 긴장감이 고조되고 미국 ...
2,2023-10-13,나흘 연속 오르던 뉴욕증시가 인플레이션 재발우려와 국채수익률 반등으로 인해 하락세로...
4,2023-10-13,나흘 연속 오르던 뉴욕증시가 인플레이션 재발우려와 국채수익률 반등으로 인해 하락세로...
7,2023-10-12,현재 미국 금융시장은 금리 하락세가 막을 내리는 거대한 변화를 겪고 있어 주식 비중...
0,2023-10-04,미국 국채 금리 급등이 국내 가계대출 금리 상승으로 이어질 것으로 전망된다. 추석 ...
1,2023-10-04,재닛 옐런 미국 재무장관이 금리가 장기간 상승세를 유지할 것이란 전망은 '기정사실'...
3,2023-09-26,"유럽중앙은행(ECB)이 최근 금리인상 종료를 시사했지만, 금리인하의 통화완화를 논하..."
5,2023-09-26,"삼성증권이 에 대한 투자의견을 '매수', 목표주가를 5900원으로 유지했다. ..."
14,2023-03-16,(서울=뉴스1) 김서연 기자 = 필리핀 중앙은행이 신종 코로나바이러스 감염증(코로나...
18,2023-03-16,“연준이 무기고에서 가장 큰 무기를 꺼내들었다” 15일(현지시간) 미국 중앙은행인...


In [9]:
pip install ekonlpy

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [10]:
import konlpy as kn
import MeCab as mc
import ekonlpy as ek
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [11]:
from ekonlpy.tag import Mecab
from ekonlpy.sentiment import MPCK
mpck = MPCK()

df = pd.read_json("news_test.json")
df = df.sort_values(by=["date"], ascending=False)

df = pd.DataFrame(df)

df['words']="" #빈 값을 가진 words column 추가

for i, row in df.iterrows() :

    try :
        tmp_df = df['content']
        #임시 데이터프레임에 불러온 기사의 본문 내용 리스트로 저장

        tokens = mpck.tokenize(tmp_df[i])
        row['words'] = '.'.join(tokens)
        df.loc[i, 'words'] = row['words']
        #eKoNLPy로 tokenizing 한 것을 데이터프레임에 column 추가

        print("success")
        if i % 100 == 0 :
            print('{}/{} : 성공'.format(i, df.shape[0]))
    except Exception as ex :
        print("fail")
        #print("{} : {}".format(i, str(ex)))

# df.to_csv("news_test.csv", mode='w')

success
success
success
success
success
0/20 : 성공
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


In [12]:
import re

# 정규표현식을 사용하여 한글 단어와 쉼표를 추출하는 함수
def extract_hangul_words(text):
    words = ' '.join(re.findall('[가-힣]+', text))  # 쉼표와 함께 한글 단어를 문자열로 반환
    return words

# 시리즈의 각 항목에 함수를 적용하여 한글 단어와 쉼표를 추출
df['words'] = df['words'].apply(extract_hangul_words)

In [13]:
df['words']

6     달러 장중 근접 하 분쟁 긴장 고조 되 만기 국채 금리 오르 시장 위험자산 회피 강...
2     연속 오르 인플레이션 재발 우려 국채 수익률 상승 인하 하락 돌아 물가 지표 크 우...
4     연속 오르 인플레이션 재발 우려 국채 수익률 상승 인하 하락 돌아 물가 지표 크 우...
7     금융시장 금리 하락 막 내리 변화 겪 주식비중 줄 채권투자 늘 포트폴리오 조정 필요...
0     국채 금리 급등 가계 대출 금리 상승 이어지 전망 되 추석 끝 채권시장 시장 영향 ...
1     금리 장기간 상승 유지 하 전망 기정사실 아니 선 그 경제 부채 상환 비용 충분히 ...
3     금리 인상 종료 시사 하 금리 인하 통화 완화 논하 아직 이르 지적 제기 되 현지 ...
5     투자의견 매수 목표가격 유지 하 대출 만기 고금리 지친 시장 부각 되 강력 투자처 ...
14    뉴스 기자 신종 코로나 바이러스 코로나 여파 금리 포인트 인하 하 있 시사 하 따르...
18    무기고 가장 크 꺼내 현지 금리 포인트 긴급 인하 하 같이 평가 하 포인트 인하 이...
17    임시 소집 하 금리 인하 코로나 확산 따르 경제 충격 완화 위하 대책 논의 하 예상...
16    시장 흔드 이슈 날카롭 시선 분석 하 장 대응 전략 제시 하 시장 신호 아침 머니 ...
15    증시 상승 하락 거듭 하 미 포인트 금리 인하 불구 여전히 불안 투자심리 반영 하 ...
19    저금리 기조 이어지 변동 주택담보대출 주택담보대출 금리 기준 되 코픽스 자금 조달 ...
11    하락 마감 하 대거 약세 보 나스닥 하락 하 현지 다우존스산업평균지수 포인트 내리 ...
12    더불 금리 상승 서민 실수요자 대출 어렵 덜 사이클 위하 변동 금리 고정 금리 갈아...
9     세계 각국 인플레이션 물가 상승 잡 위하 빠르 속도 금리 오르 도대체 얼마나 금리 ...
10    새해 주 설 명절 전후 쏟 은행 특판 특별 판매 예금 자취 감췄 은행 유동성

In [14]:
# 'column_name'이라는 열을 제거
new_df = df.drop('content', axis=1)  # axis=1은 열을 의미함
new_df.columns

Index(['date', 'words'], dtype='object')

In [15]:
import pandas as pd

# Date 열을 datetime으로 변환하고 인덱스로 설정
result['date'] = pd.to_datetime(result['date'])
result.set_index('date', inplace=True)

new_df['date'] = pd.to_datetime(new_df['date'])
new_df.set_index('date', inplace=True)

# 중복된 인덱스 제거
result = result[~result.index.duplicated()]
new_df = new_df[~new_df.index.duplicated()]

# 동일한 날짜에 대해 열을 합치기
merged_df = pd.concat([result, new_df], axis=1)

# 'date' 열을 다시 열로 변환
merged_df.reset_index(inplace=True)

# 결과 출력
print(merged_df.columns)
print(merged_df)

Index(['date', '종가', 'label', 'words'], dtype='object')
         date      종가 label                                              words
0  2023-01-01     NaN   NaN  채권시장 양상 보 예상 되 금리 인상 멈추 채권 금리 안정화 되 채권 가격 오르 전...
1  2023-01-02  3.2910     0                                                NaN
2  2023-01-10     NaN   NaN  세계 각국 인플레이션 물가 상승 잡 위하 빠르 속도 금리 오르 도대체 얼마나 금리 ...
3  2023-01-13  3.4270    매파  더불 금리 상승 서민 실수요자 대출 어렵 덜 사이클 위하 변동 금리 고정 금리 갈아...
4  2023-01-14     NaN   NaN  하락 마감 하 대거 약세 보 나스닥 하락 하 현지 다우존스산업평균지수 포인트 내리 ...
5  2023-01-16  3.3940    매파                                                NaN
6  2023-01-17  3.3690    매파                                                NaN
7  2023-01-18  3.3650    매파                                                NaN
8  2023-01-19  3.3870    매파                                                NaN
9  2023-01-20  3.4800    매파                                                NaN
10 2023-01-25  3.4350    매파                                                

In [26]:
# 'label' 칼럼의 값이 '매파'인 행과 '비둘기파'인 행으로 데이터프레임 분리
df_A = merged_df[merged_df['label'] == '매파']
df_B = merged_df[merged_df['label'] == '비둘기파']

# Nan인 행 제거
df_A = df_A.dropna()
df_B = df_B.dropna()

# 필요없는 열 제거 
df_A = df_A.drop(['date', '종가'], axis=1)
df_B = df_B.drop(['date', '종가'], axis=1)

word_df = pd.concat([df_A,df_B],axis=0)
x_tain = word_df["words"]
y_tain = word_df["label"]

In [23]:
df_A

,label,words
3,매파,더불 금리 상승 서민 실수요자 대출 어렵 덜 사이클 위하 변동 금리 고정 금리 갈아...
15,매파,뉴스 기자 신종 코로나 바이러스 코로나 여파 금리 포인트 인하 하 있 시사 하 따르...
19,매파,금리 인상 종료 시사 하 금리 인하 통화 완화 논하 아직 이르 지적 제기 되 현지 ...


In [24]:
df_B

,label,words
21,비둘기파,국채 금리 급등 가계 대출 금리 상승 이어지 전망 되 추석 끝 채권시장 시장 영향 ...
23,비둘기파,금융시장 금리 하락 막 내리 변화 겪 주식비중 줄 채권투자 늘 포트폴리오 조정 필요...
24,비둘기파,연속 오르 인플레이션 재발 우려 국채 수익률 상승 인하 하락 돌아 물가 지표 크 우...
28,비둘기파,달러 장중 근접 하 분쟁 긴장 고조 되 만기 국채 금리 오르 시장 위험자산 회피 강...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from ekonlpy import Mecab
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer 설정
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=0.0)  # max_df 값을 1.0으로 설정하여 모든 단어를 포함하도록 함

# 문서를 벡터화
tfidf_matrix = tfidf_vectorizer.fit_transform(x_tain)

# accuracy_score(y_test, y_pred)
# precision_score(y_test, y_pred)
# recall_score(y_test, y_pred)
# f1_score(y_test, y_pred)

#모델 훈련및 결과출력
# def Model_test(word_list,file_text):
    # 1. 데이터 준비
X_train = x_tain
y_train = y_tain
print("학습물을 다 읽었습니다")
# 2. 특성 추출 (형태소 분석 및 n-그램)
def taging(text):
    return text.split()
#tagger = taging(text)
mecab_tokenize = lambda text: taging(text)
vectorizer = CountVectorizer(tokenizer=mecab_tokenize, ngram_range=(1, 5), min_df=15)
X_train_counts = vectorizer.fit_transform(X_train)
print("벡터라이즈를 생성하였습니다")
print("학습을 시작합니다")
# 3. 나이브 베이즈 모델 학습
clf = MultinomialNB()
clf.fit(X_train_counts, y_train)
print("학습을 시작합니다")
print(clf.score(X_train_counts, y_train))
#4. 새로운 입력 예측
X_test = [file_text]
X_test_counts = vectorizer.transform(X_test)
predicted_labels = clf.predict(X_test_counts)
print(predicted_labels)

In [30]:
word_df

,label,words
3,매파,더불 금리 상승 서민 실수요자 대출 어렵 덜 사이클 위하 변동 금리 고정 금리 갈아...
15,매파,뉴스 기자 신종 코로나 바이러스 코로나 여파 금리 포인트 인하 하 있 시사 하 따르...
19,매파,금리 인상 종료 시사 하 금리 인하 통화 완화 논하 아직 이르 지적 제기 되 현지 ...
21,비둘기파,국채 금리 급등 가계 대출 금리 상승 이어지 전망 되 추석 끝 채권시장 시장 영향 ...
23,비둘기파,금융시장 금리 하락 막 내리 변화 겪 주식비중 줄 채권투자 늘 포트폴리오 조정 필요...
24,비둘기파,연속 오르 인플레이션 재발 우려 국채 수익률 상승 인하 하락 돌아 물가 지표 크 우...
28,비둘기파,달러 장중 근접 하 분쟁 긴장 고조 되 만기 국채 금리 오르 시장 위험자산 회피 강...


In [31]:
def generate_ngrams(s, n):
    tokens = s.split()
    ngrams = zip(*[tokens[i:] for i in range(n)])
    
    return ["_".join(ngram) for ngram in ngrams]

In [32]:
word_df['1_ngrams'] = word_df["words"].iloc[0:].apply(lambda row: (" ".join(generate_ngrams(row, 1))))
print("1 그램 생성")
word_df['2_ngrams'] = word_df["words"].iloc[0:].apply(lambda row: (" ".join(generate_ngrams(row, 2))))
print("2 그램 생성")
word_df['3_ngrams'] = word_df["words"].iloc[0:].apply(lambda row: (" ".join(generate_ngrams(row, 3))))
print("3 그램 생성")
word_df['4_ngrams'] = word_df["words"].iloc[0:].apply(lambda row: (" ".join(generate_ngrams(row, 4))))
print("4 그램 생성")
word_df['5_ngrams'] = word_df["words"].iloc[0:].apply(lambda row: (" ".join(generate_ngrams(row, 5))))
print("5 그램 생성")
word_df.to_csv('data_tokens_ngram_1_5.csv', index=False)

1 그램 생성
2 그램 생성
3 그램 생성
4 그램 생성
5 그램 생성


In [33]:
token_df = pd.read_csv("./data_tokens_ngram_1_5.csv")
token_df

,label,words,1_ngrams,2_ngrams,3_ngrams,4_ngrams,5_ngrams
0,매파,더불 금리 상승 서민 실수요자 대출 어렵 덜 사이클 위하 변동 금리 고정 금리 갈아...,더불 금리 상승 서민 실수요자 대출 어렵 덜 사이클 위하 변동 금리 고정 금리 갈아...,더불_금리 금리_상승 상승_서민 서민_실수요자 실수요자_대출 대출_어렵 어렵_덜 덜...,더불_금리_상승 금리_상승_서민 상승_서민_실수요자 서민_실수요자_대출 실수요자_대...,더불_금리_상승_서민 금리_상승_서민_실수요자 상승_서민_실수요자_대출 서민_실수요...,더불_금리_상승_서민_실수요자 금리_상승_서민_실수요자_대출 상승_서민_실수요자_대...
1,매파,뉴스 기자 신종 코로나 바이러스 코로나 여파 금리 포인트 인하 하 있 시사 하 따르...,뉴스 기자 신종 코로나 바이러스 코로나 여파 금리 포인트 인하 하 있 시사 하 따르...,뉴스_기자 기자_신종 신종_코로나 코로나_바이러스 바이러스_코로나 코로나_여파 여파...,뉴스_기자_신종 기자_신종_코로나 신종_코로나_바이러스 코로나_바이러스_코로나 바이...,뉴스_기자_신종_코로나 기자_신종_코로나_바이러스 신종_코로나_바이러스_코로나 코로...,뉴스_기자_신종_코로나_바이러스 기자_신종_코로나_바이러스_코로나 신종_코로나_바이...
2,매파,금리 인상 종료 시사 하 금리 인하 통화 완화 논하 아직 이르 지적 제기 되 현지 ...,금리 인상 종료 시사 하 금리 인하 통화 완화 논하 아직 이르 지적 제기 되 현지 ...,금리_인상 인상_종료 종료_시사 시사_하 하_금리 금리_인하 인하_통화 통화_완화 ...,금리_인상_종료 인상_종료_시사 종료_시사_하 시사_하_금리 하_금리_인하 금리_인...,금리_인상_종료_시사 인상_종료_시사_하 종료_시사_하_금리 시사_하_금리_인하 하...,금리_인상_종료_시사_하 인상_종료_시사_하_금리 종료_시사_하_금리_인하 시사_하...
3,비둘기파,국채 금리 급등 가계 대출 금리 상승 이어지 전망 되 추석 끝 채권시장 시장 영향 ...,국채 금리 급등 가계 대출 금리 상승 이어지 전망 되 추석 끝 채권시장 시장 영향 ...,국채_금리 금리_급등 급등_가계 가계_대출 대출_금리 금리_상승 상승_이어지 이어지...,국채_금리_급등 금리_급등_가계 급등_가계_대출 가계_대출_금리 대출_금리_상승 금...,국채_금리_급등_가계 금리_급등_가계_대출 급등_가계_대출_금리 가계_대출_금리_상...,국채_금리_급등_가계_대출 금리_급등_가계_대출_금리 급등_가계_대출_금리_상승 가...
4,비둘기파,금융시장 금리 하락 막 내리 변화 겪 주식비중 줄 채권투자 늘 포트폴리오 조정 필요...,금융시장 금리 하락 막 내리 변화 겪 주식비중 줄 채권투자 늘 포트폴리오 조정 필요...,금융시장_금리 금리_하락 하락_막 막_내리 내리_변화 변화_겪 겪_주식비중 주식비중...,금융시장_금리_하락 금리_하락_막 하락_막_내리 막_내리_변화 내리_변화_겪 변화_...,금융시장_금리_하락_막 금리_하락_막_내리 하락_막_내리_변화 막_내리_변화_겪 내...,금융시장_금리_하락_막_내리 금리_하락_막_내리_변화 하락_막_내리_변화_겪 막_내...
5,비둘기파,연속 오르 인플레이션 재발 우려 국채 수익률 상승 인하 하락 돌아 물가 지표 크 우...,연속 오르 인플레이션 재발 우려 국채 수익률 상승 인하 하락 돌아 물가 지표 크 우...,연속_오르 오르_인플레이션 인플레이션_재발 재발_우려 우려_국채 국채_수익률 수익률...,연속_오르_인플레이션 오르_인플레이션_재발 인플레이션_재발_우려 재발_우려_국채 우...,연속_오르_인플레이션_재발 오르_인플레이션_재발_우려 인플레이션_재발_우려_국채 재...,연속_오르_인플레이션_재발_우려 오르_인플레이션_재발_우려_국채 인플레이션_재발_우...
6,비둘기파,달러 장중 근접 하 분쟁 긴장 고조 되 만기 국채 금리 오르 시장 위험자산 회피 강...,달러 장중 근접 하 분쟁 긴장 고조 되 만기 국채 금리 오르 시장 위험자산 회피 강...,달러_장중 장중_근접 근접_하 하_분쟁 분쟁_긴장 긴장_고조 고조_되 되_만기 만기...,달러_장중_근접 장중_근접_하 근접_하_분쟁 하_분쟁_긴장 분쟁_긴장_고조 긴장_고...,달러_장중_근접_하 장중_근접_하_분쟁 근접_하_분쟁_긴장 하_분쟁_긴장_고조 분쟁...,달러_장중_근접_하_분쟁 장중_근접_하_분쟁_긴장 근접_하_분쟁_긴장_고조 하_분쟁...


In [34]:
token_df.isnull().sum()

label       0
words       0
1_ngrams    0
2_ngrams    0
3_ngrams    0
4_ngrams    0
5_ngrams    0
dtype: int64

In [35]:
token_df[token_df['2_ngrams'].isnull()]

,label,words,1_ngrams,2_ngrams,3_ngrams,4_ngrams,5_ngrams


In [36]:
drop_token_df = token_df.dropna(axis=0)
drop_token_df

,label,words,1_ngrams,2_ngrams,3_ngrams,4_ngrams,5_ngrams
0,매파,더불 금리 상승 서민 실수요자 대출 어렵 덜 사이클 위하 변동 금리 고정 금리 갈아...,더불 금리 상승 서민 실수요자 대출 어렵 덜 사이클 위하 변동 금리 고정 금리 갈아...,더불_금리 금리_상승 상승_서민 서민_실수요자 실수요자_대출 대출_어렵 어렵_덜 덜...,더불_금리_상승 금리_상승_서민 상승_서민_실수요자 서민_실수요자_대출 실수요자_대...,더불_금리_상승_서민 금리_상승_서민_실수요자 상승_서민_실수요자_대출 서민_실수요...,더불_금리_상승_서민_실수요자 금리_상승_서민_실수요자_대출 상승_서민_실수요자_대...
1,매파,뉴스 기자 신종 코로나 바이러스 코로나 여파 금리 포인트 인하 하 있 시사 하 따르...,뉴스 기자 신종 코로나 바이러스 코로나 여파 금리 포인트 인하 하 있 시사 하 따르...,뉴스_기자 기자_신종 신종_코로나 코로나_바이러스 바이러스_코로나 코로나_여파 여파...,뉴스_기자_신종 기자_신종_코로나 신종_코로나_바이러스 코로나_바이러스_코로나 바이...,뉴스_기자_신종_코로나 기자_신종_코로나_바이러스 신종_코로나_바이러스_코로나 코로...,뉴스_기자_신종_코로나_바이러스 기자_신종_코로나_바이러스_코로나 신종_코로나_바이...
2,매파,금리 인상 종료 시사 하 금리 인하 통화 완화 논하 아직 이르 지적 제기 되 현지 ...,금리 인상 종료 시사 하 금리 인하 통화 완화 논하 아직 이르 지적 제기 되 현지 ...,금리_인상 인상_종료 종료_시사 시사_하 하_금리 금리_인하 인하_통화 통화_완화 ...,금리_인상_종료 인상_종료_시사 종료_시사_하 시사_하_금리 하_금리_인하 금리_인...,금리_인상_종료_시사 인상_종료_시사_하 종료_시사_하_금리 시사_하_금리_인하 하...,금리_인상_종료_시사_하 인상_종료_시사_하_금리 종료_시사_하_금리_인하 시사_하...
3,비둘기파,국채 금리 급등 가계 대출 금리 상승 이어지 전망 되 추석 끝 채권시장 시장 영향 ...,국채 금리 급등 가계 대출 금리 상승 이어지 전망 되 추석 끝 채권시장 시장 영향 ...,국채_금리 금리_급등 급등_가계 가계_대출 대출_금리 금리_상승 상승_이어지 이어지...,국채_금리_급등 금리_급등_가계 급등_가계_대출 가계_대출_금리 대출_금리_상승 금...,국채_금리_급등_가계 금리_급등_가계_대출 급등_가계_대출_금리 가계_대출_금리_상...,국채_금리_급등_가계_대출 금리_급등_가계_대출_금리 급등_가계_대출_금리_상승 가...
4,비둘기파,금융시장 금리 하락 막 내리 변화 겪 주식비중 줄 채권투자 늘 포트폴리오 조정 필요...,금융시장 금리 하락 막 내리 변화 겪 주식비중 줄 채권투자 늘 포트폴리오 조정 필요...,금융시장_금리 금리_하락 하락_막 막_내리 내리_변화 변화_겪 겪_주식비중 주식비중...,금융시장_금리_하락 금리_하락_막 하락_막_내리 막_내리_변화 내리_변화_겪 변화_...,금융시장_금리_하락_막 금리_하락_막_내리 하락_막_내리_변화 막_내리_변화_겪 내...,금융시장_금리_하락_막_내리 금리_하락_막_내리_변화 하락_막_내리_변화_겪 막_내...
5,비둘기파,연속 오르 인플레이션 재발 우려 국채 수익률 상승 인하 하락 돌아 물가 지표 크 우...,연속 오르 인플레이션 재발 우려 국채 수익률 상승 인하 하락 돌아 물가 지표 크 우...,연속_오르 오르_인플레이션 인플레이션_재발 재발_우려 우려_국채 국채_수익률 수익률...,연속_오르_인플레이션 오르_인플레이션_재발 인플레이션_재발_우려 재발_우려_국채 우...,연속_오르_인플레이션_재발 오르_인플레이션_재발_우려 인플레이션_재발_우려_국채 재...,연속_오르_인플레이션_재발_우려 오르_인플레이션_재발_우려_국채 인플레이션_재발_우...
6,비둘기파,달러 장중 근접 하 분쟁 긴장 고조 되 만기 국채 금리 오르 시장 위험자산 회피 강...,달러 장중 근접 하 분쟁 긴장 고조 되 만기 국채 금리 오르 시장 위험자산 회피 강...,달러_장중 장중_근접 근접_하 하_분쟁 분쟁_긴장 긴장_고조 고조_되 되_만기 만기...,달러_장중_근접 장중_근접_하 근접_하_분쟁 하_분쟁_긴장 분쟁_긴장_고조 긴장_고...,달러_장중_근접_하 장중_근접_하_분쟁 근접_하_분쟁_긴장 하_분쟁_긴장_고조 분쟁...,달러_장중_근접_하_분쟁 장중_근접_하_분쟁_긴장 근접_하_분쟁_긴장_고조 하_분쟁...
